In [220]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import gensim
import pandas as pd
from collections import defaultdict

from tqdm.auto import tqdm
tqdm.pandas()

import plotly.express as px

In [236]:
all_data = pd.read_csv('./manifesto_articles_preprocessed.csv')

In [238]:
all_data.words = all_data.words.progress_apply(eval)

  0%|          | 0/69935 [00:00<?, ?it/s]

In [239]:
#all_data.words = all_data.words.progress_apply(lambda x: [a for a in x if a not in ["→","”","“"]])

In [112]:
dict_party = {'Socialist Party':'PS',
              'Europe Ecology - The Greens':'EELV',
              'The Republicans':'RPR/UMP/LR',
              'National Front':'FN',
              'Democratic Movement':'MoDem/EM',
              'Republic Onwards!':'MoDem/EM',
              'Left Front':'PCF/PG/LFI',
              'Indomitable France':'PCF/PG/LFI',
              'French Communist Party':'PCF/PG/LFI',
              'Le Monde':'Le Monde',
              'Le Figaro':'Le Figaro'}

In [114]:
dict_color = {'PS': 'pink', 
              'MoDem/EM': 'orange', 
              'RPR/UMP/LR': 'blue',
              'FN':'darkslateblue',
              'EELV':'green',
              'PCF/PG/LFI':'red',
              'Le Monde':'black',
              'Le Figaro':'cyan'}

In [115]:
all_data.partyname = all_data.partyname.apply(lambda x: dict_party[x])

In [116]:
df2002 = all_data[all_data['fichier']==1].reset_index(drop=True)
df2007 = all_data[all_data['fichier']==2].reset_index(drop=True)
df2012 = all_data[all_data['fichier']==3].reset_index(drop=True)
df2017 = all_data[all_data['fichier']==4].reset_index(drop=True)

## Doc2Vec

In [226]:
def doc2vec(df):
    corpus = df.apply(lambda row: gensim.models.doc2vec.TaggedDocument(row['words'], [row['index']]),axis=1)
    model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=10)
    model.build_vocab(corpus)
    model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)
    
    manifesto_ids = df.loc[~df['partyname'].isin(['Le Monde','Le Figaro']),['index','partyname']].reset_index(drop=True)
    for i in range(manifesto_ids.shape[0]):
        df[manifesto_ids.loc[i,'partyname']] = model.dv.cosine_similarities(model.dv.vectors[manifesto_ids.loc[i,'index']],model.dv.vectors)
    sims = df.loc[df['partyname'].isin(['Le Monde', 'Le Figaro'])].drop('index',axis=1).groupby(['partyname','annee']).mean().reset_index()
    return sims

In [227]:
sims2002 = doc2vec(df2002)

2021-12-01 14:35:53,941 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3)', 'datetime': '2021-12-01T14:35:53.941802', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
2021-12-01 14:35:53,942 : INFO : collecting all words and their counts
2021-12-01 14:35:53,942 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-12-01 14:35:54,228 : INFO : PROGRESS: at example #10000, processed 1952759 words (6854310/s), 33963 word types, 0 tags
2021-12-01 14:35:54,428 : INFO : collected 42673 word types and 18842 unique tags from a corpus of 18842 examples and 3372991 words
2021-12-01 14:35:54,429 : INFO : Creating a fresh vocabulary
2021-12-01 14:35:54,530 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 26897 unique words (63.03048766198767%% of original 42673, drops 15776)', 'datetime': '2021-12-01T14:35:54.

2021-12-01 14:36:22,882 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-12-01 14:36:22,885 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-12-01 14:36:22,893 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-01 14:36:22,894 : INFO : EPOCH - 8 : training on 3372991 raw words (3158230 effective words) took 3.2s, 982355 effective words/s
2021-12-01 14:36:23,908 : INFO : EPOCH 9 - PROGRESS: at 28.04% examples, 1003125 words/s, in_qsize 6, out_qsize 0
2021-12-01 14:36:24,916 : INFO : EPOCH 9 - PROGRESS: at 59.53% examples, 999711 words/s, in_qsize 6, out_qsize 0
2021-12-01 14:36:25,919 : INFO : EPOCH 9 - PROGRESS: at 92.96% examples, 989214 words/s, in_qsize 6, out_qsize 0
2021-12-01 14:36:26,093 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-12-01 14:36:26,096 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-12-01 14:36:26,116 : INFO : worker thread finished; awaiting fi

In [230]:
sims2007 = doc2vec(df2007)

2021-12-01 14:37:46,023 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3)', 'datetime': '2021-12-01T14:37:46.023549', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
2021-12-01 14:37:46,024 : INFO : collecting all words and their counts
2021-12-01 14:37:46,025 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-12-01 14:37:46,340 : INFO : PROGRESS: at example #10000, processed 1843854 words (5867980/s), 36533 word types, 0 tags
2021-12-01 14:37:46,553 : INFO : PROGRESS: at example #20000, processed 3138074 words (6104579/s), 44189 word types, 0 tags
2021-12-01 14:37:46,627 : INFO : collected 46137 word types and 23135 unique tags from a corpus of 23135 examples and 3519387 words
2021-12-01 14:37:46,628 : INFO : Creating a fresh vocabulary
2021-12-01 14:37:46,743 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min

2021-12-01 14:38:20,032 : INFO : EPOCH 7 - PROGRESS: at 76.91% examples, 875012 words/s, in_qsize 6, out_qsize 0
2021-12-01 14:38:20,757 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-12-01 14:38:20,769 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-12-01 14:38:20,773 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-01 14:38:20,773 : INFO : EPOCH - 7 : training on 3519387 raw words (3283879 effective words) took 3.8s, 868879 effective words/s
2021-12-01 14:38:21,797 : INFO : EPOCH 8 - PROGRESS: at 19.47% examples, 843221 words/s, in_qsize 5, out_qsize 0
2021-12-01 14:38:22,822 : INFO : EPOCH 8 - PROGRESS: at 42.10% examples, 826719 words/s, in_qsize 5, out_qsize 0
2021-12-01 14:38:23,824 : INFO : EPOCH 8 - PROGRESS: at 73.41% examples, 840045 words/s, in_qsize 5, out_qsize 0
2021-12-01 14:38:24,762 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-12-01 14:38:24,777 : INFO : worker thread 

In [231]:
sims2012 = doc2vec(df2012)

2021-12-01 14:38:32,879 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3)', 'datetime': '2021-12-01T14:38:32.879440', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
2021-12-01 14:38:32,880 : INFO : collecting all words and their counts
2021-12-01 14:38:32,880 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-12-01 14:38:33,256 : INFO : PROGRESS: at example #10000, processed 2154185 words (5738401/s), 37805 word types, 0 tags
2021-12-01 14:38:33,547 : INFO : PROGRESS: at example #20000, processed 3635309 words (5104762/s), 46989 word types, 0 tags
2021-12-01 14:38:33,561 : INFO : collected 47301 word types and 20555 unique tags from a corpus of 20555 examples and 3669554 words
2021-12-01 14:38:33,562 : INFO : Creating a fresh vocabulary
2021-12-01 14:38:33,721 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min

2021-12-01 14:39:01,701 : INFO : EPOCH 8 - PROGRESS: at 46.86% examples, 982116 words/s, in_qsize 5, out_qsize 0
2021-12-01 14:39:02,706 : INFO : EPOCH 8 - PROGRESS: at 78.32% examples, 957541 words/s, in_qsize 6, out_qsize 0
2021-12-01 14:39:03,313 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-12-01 14:39:03,320 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-12-01 14:39:03,332 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-01 14:39:03,333 : INFO : EPOCH - 8 : training on 3669554 raw words (3444468 effective words) took 3.6s, 946574 effective words/s
2021-12-01 14:39:04,340 : INFO : EPOCH 9 - PROGRESS: at 20.10% examples, 915062 words/s, in_qsize 6, out_qsize 0
2021-12-01 14:39:05,346 : INFO : EPOCH 9 - PROGRESS: at 44.39% examples, 944977 words/s, in_qsize 6, out_qsize 0
2021-12-01 14:39:06,355 : INFO : EPOCH 9 - PROGRESS: at 76.03% examples, 931784 words/s, in_qsize 5, out_qsize 0
2021-12-01 14:39:07,039 :

In [232]:
sims2017 = doc2vec(df2017)

2021-12-01 14:39:13,231 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3)', 'datetime': '2021-12-01T14:39:13.231139', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}
2021-12-01 14:39:13,231 : INFO : collecting all words and their counts
2021-12-01 14:39:13,235 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-12-01 14:39:13,750 : INFO : collected 35558 word types and 7403 unique tags from a corpus of 7403 examples and 1447555 words
2021-12-01 14:39:13,751 : INFO : Creating a fresh vocabulary
2021-12-01 14:39:13,856 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 21573 unique words (60.669891444963156%% of original 35558, drops 13985)', 'datetime': '2021-12-01T14:39:13.856454', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 12:59:45) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x8

In [243]:
sims = pd.concat([sims2002,sims2007,sims2012,sims2017])

In [244]:
sims.fichier = sims.fichier.replace({1:2002,2:2007,3:2012,4:2017})

In [249]:
fig = px.line(sims,
              x='annee',
              y=sims.columns[3:],
              labels={'partyname':'Journal','variable':'Party','value':'Cosine Similarity','annee':'year', 'fichier':'manifesto'},
              title='Doc2Vec',
              facet_row='partyname',
              facet_col='fichier',
              color_discrete_map=dict_color
       )
fig.update_xaxes(matches=None)
fig.show()